# T1548.001 - Setuid and Setgid
An adversary may abuse configurations where an application has the setuid or setgid bits set in order to get code running in a different (and possibly more privileged) user’s context. On Linux or macOS, when the setuid or setgid bits are set for an application binary, the application will run with the privileges of the owning user or group respectively.(Citation: setuid man page) Normally an application is run in the current user’s context, regardless of which user or group owns the application. However, there are instances where programs need to be executed in an elevated context to function properly, but the user running them may not have the specific required privileges.

Instead of creating an entry in the sudoers file, which must be done by root, any user can specify the setuid or setgid flag to be set for their own applications (i.e. [Linux and Mac File and Directory Permissions Modification](https://attack.mitre.org/techniques/T1222/002)). The <code>chmod</code> command can set these bits with bitmasking, <code>chmod 4777 [file]</code> or via shorthand naming, <code>chmod u+s [file]</code>. This will enable the setuid bit. To enable the setgid bit, <code>chmod 2775</code> and <code>chmod g+s</code> can be used.

Adversaries can use this mechanism on their own malware to make sure they're able to execute in elevated contexts in the future.(Citation: OSX Keydnap malware) This abuse is often part of a "shell escape" or other actions to bypass an execution environment with restricted permissions.

Alternatively, adversaries may choose to find and target vulnerable binaries with the setuid or setgid bits already enabled (i.e. [File and Directory Discovery](https://attack.mitre.org/techniques/T1083)). The setuid and setguid bits are indicated with an "s" instead of an "x" when viewing a file's attributes via <code>ls -l</code>. The <code>find</code> command can also be used to search for such files. For example, <code>find / -perm +4000 2>/dev/null</code> can be used to find files with setuid set and <code>find / -perm +2000 2>/dev/null</code> may be used for setgid. Binaries that have these bits set may then be abused by adversaries.(Citation: GTFOBins Suid)

## Atomic Tests

### Atomic Test #1 - Make and modify binary from C sourceMake, change owner, and change file attributes on a C source code file
**Supported Platforms:** macos, linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
cp PathToAtomicsFolder/T1548.001/src/hello.c /tmp/hello.c
sudo chown root /tmp/hello.c
sudo make /tmp/hello
sudo chown root /tmp/hello
sudo chmod u+s /tmp/hello
/tmp/hello
```

In [ ]:
Invoke-AtomicTest T1548.001 -TestNumbers 1

#### Cleanup: 
```sh
sudo rm /tmp/hello
sudo rm /tmp/hello.c
```

In [ ]:
Invoke-AtomicTest T1548.001 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Make and modify binary from C source (freebsd)Make, change owner, and change file attributes on a C source code file
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
cp PathToAtomicsFolder/T1548.001/src/hello.c /tmp/hello.c
chown root /tmp/hello.c
make /tmp/hello
chown root /tmp/hello
chmod u+s /tmp/hello
/tmp/hello
```

In [ ]:
Invoke-AtomicTest T1548.001 -TestNumbers 2

#### Cleanup: 
```sh
rm /tmp/hello
rm /tmp/hello.c
```

In [ ]:
Invoke-AtomicTest T1548.001 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Set a SetUID flag on fileThis test sets the SetUID flag on a file in FreeBSD.
**Supported Platforms:** macos, linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
sudo touch /tmp/evilBinary
sudo chown root /tmp/evilBinary
sudo chmod u+xs /tmp/evilBinary
```

In [ ]:
Invoke-AtomicTest T1548.001 -TestNumbers 3

#### Cleanup: 
```sh
sudo rm /tmp/evilBinary
```

In [ ]:
Invoke-AtomicTest T1548.001 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Set a SetUID flag on file (freebsd)This test sets the SetUID flag on a file in FreeBSD.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
touch /tmp/evilBinary
chown root /tmp/evilBinary
chmod u+xs /tmp/evilBinary
```

In [ ]:
Invoke-AtomicTest T1548.001 -TestNumbers 4

#### Cleanup: 
```sh
rm /tmp/evilBinary
```

In [ ]:
Invoke-AtomicTest T1548.001 -TestNumbers 4 -Cleanup

### Atomic Test #5 - Set a SetGID flag on fileThis test sets the SetGID flag on a file in Linux and macOS.
**Supported Platforms:** macos, linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
sudo touch /tmp/evilBinary
sudo chown root /tmp/evilBinary
sudo chmod g+xs /tmp/evilBinary
```

In [ ]:
Invoke-AtomicTest T1548.001 -TestNumbers 5

#### Cleanup: 
```sh
sudo rm /tmp/evilBinary
```

In [ ]:
Invoke-AtomicTest T1548.001 -TestNumbers 5 -Cleanup

### Atomic Test #6 - Set a SetGID flag on file (freebsd)This test sets the SetGID flag on a file in FreeBSD.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
touch /tmp/evilBinary
chown root /tmp/evilBinary
chmod g+xs /tmp/evilBinary
```

In [ ]:
Invoke-AtomicTest T1548.001 -TestNumbers 6

#### Cleanup: 
```sh
rm /tmp/evilBinary
```

In [ ]:
Invoke-AtomicTest T1548.001 -TestNumbers 6 -Cleanup

### Atomic Test #7 - Make and modify capabilities of a binaryMake and modify [capabilities](https://man7.org/linux/man-pages/man7/capabilities.7.html) of a C source code file.
The binary doesn't have to modify the UID, but the binary is given the capability to arbitrarily modify it at any time with `setuid(0)`.
Without being owned by root, the binary can set the UID to 0.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
cp PathToAtomicsFolder/T1548.001/src/cap.c /tmp/cap.c
make /tmp/cap
sudo setcap cap_setuid=ep /tmp/cap
/tmp/cap
```

In [ ]:
Invoke-AtomicTest T1548.001 -TestNumbers 7

#### Cleanup: 
```sh
rm /tmp/cap
rm /tmp/cap.c
```

In [ ]:
Invoke-AtomicTest T1548.001 -TestNumbers 7 -Cleanup

### Atomic Test #8 - Provide the SetUID capability to a fileThis test gives a file the capability to set UID without using flags.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
touch /tmp/evilBinary
sudo setcap cap_setuid=ep /tmp/evilBinary
```

In [ ]:
Invoke-AtomicTest T1548.001 -TestNumbers 8

#### Cleanup: 
```sh
rm /tmp/evilBinary
```

In [ ]:
Invoke-AtomicTest T1548.001 -TestNumbers 8 -Cleanup

### Atomic Test #9 - Do reconnaissance for files that have the setuid bit setThis test simulates a command that can be run to enumerate files that have the setuid bit set
**Supported Platforms:** linux#### Attack Commands: Run with `sh`
```sh
find /usr/bin -perm -4000
```

In [ ]:
Invoke-AtomicTest T1548.001 -TestNumbers 9

### Atomic Test #10 - Do reconnaissance for files that have the setgid bit setThis test simulates a command that can be run to enumerate files that have the setgid bit set
**Supported Platforms:** linux#### Attack Commands: Run with `sh`
```sh
find /usr/bin -perm -2000
```

In [ ]:
Invoke-AtomicTest T1548.001 -TestNumbers 10

## Detection
Monitor the file system for files that have the setuid or setgid bits set. Monitor for execution of utilities, like chmod, and their command-line arguments to look for setuid or setguid bits being set.